This notebook shows how to use a `GraphCreator` instance in a recommendation pipeline to easily produce the top recommendations and display their predicted order (before/after)

In [12]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append('../utils/')

import pickle
import numpy as np
import pandas as pd

from GraphAPI import GraphCreator
from RecommenderPipeline import Recommender

from sklearn.preprocessing import normalize, StandardScaler, Normalizer, RobustScaler, MinMaxScaler, MaxAbsScaler


%aimport GraphAPI
%aimport RecommenderPipeline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load in Models

In [13]:
with open("../models/rf_classifier_v2_normalized.pkl", "rb") as model:
    rf_v2_classifier = pickle.load(model)
    
with open("../models/rf_classifier_v3_normalized_714.pkl", "rb") as model:
    rf_v3_classifier = pickle.load(model)    
    
with open("../models/rf_classifier_v4_732.pkl", "rb") as model:
    rf_v4_classifier = pickle.load(model)    
    
with open("../models/xg_model_semisupervised_v2.pkl", "rb") as model:
    xg_classifier = pickle.load(model)

# Initialize `GraphCreator` Instance

After initialization, pass as an argument to a new recommender instance

In [14]:
gc = GraphCreator("Function (music)", include_see_also=False, max_recursive_requests=50)
print("Layer 1 nodes:", len(gc.next_links))
rec = Recommender(gc)

Layer 1 nodes: 178


# Fit the Recommender 

In [15]:
rec.fit(scaler=Normalizer)

# Make Predictions
Pass in your model to make predictions on the data

In [16]:
rec.predict(rf_v2_classifier)
# rec.predict(xg_classifier)

# Format the Results
Will return as a dictionary containing the entry node and the predictions of the top articles.

In [17]:
rec.format_results()

{'entry': 'Function (music)',
 'decision_threshold': 0.4700000000000002,
 'predictions': [{'node': 'Predominant chord',
   'similarity_rank': 1.398765672066976,
   'label_proba': [0.5912831721842139, 0.40871682781578605],
   'position': 'after'},
  {'node': 'Parallel and counter parallel',
   'similarity_rank': 1.3448070423531229,
   'label_proba': [0.5044086598611277, 0.49559134013887224],
   'position': 'after'},
  {'node': 'Supertonic',
   'similarity_rank': 1.2579175906944031,
   'label_proba': [0.39913355831686353, 0.6008664416831366],
   'position': 'before'},
  {'node': 'Subtonic',
   'similarity_rank': 1.2358160086024788,
   'label_proba': [0.3950173794341712, 0.604982620565829],
   'position': 'before'},
  {'node': 'Secondary chord',
   'similarity_rank': 1.1755284426180421,
   'label_proba': [0.4833263104117881, 0.5166736895882118],
   'position': 'after'},
  {'node': 'Mediant',
   'similarity_rank': 1.1408017629506264,
   'label_proba': [0.40164189165019687, 0.59835810834980

# Optional: Format as DataFrame for Easy Viewing

In [21]:
formatted_results = rec.format_results(0.47)

recommendations = pd.DataFrame(formatted_results['predictions'])
print(recommendations.position.value_counts())
print("Decision Threshold:", round(formatted_results['decision_threshold'], 2))
recommendations[['node', 'position', "label_proba"]]

after     59
before    40
Name: position, dtype: int64
Decision Threshold: 0.47


,node,position,label_proba
0,Predominant chord,after,"[0.5912831721842139, 0.40871682781578605]"
1,Parallel and counter parallel,after,"[0.5044086598611277, 0.49559134013887224]"
2,Supertonic,before,"[0.39913355831686353, 0.6008664416831366]"
3,Subtonic,before,"[0.3950173794341712, 0.604982620565829]"
4,Secondary chord,after,"[0.4833263104117881, 0.5166736895882118]"
5,Mediant,before,"[0.40164189165019687, 0.5983581083498033]"
6,Terzschritt,after,"[0.4879287851990853, 0.5120712148009148]"
7,Tonicization,after,"[0.5320064413156713, 0.4679935586843286]"
8,Submediant,before,"[0.4304137916977266, 0.5695862083022735]"
9,Leading-tone,after,"[0.5249145510742239, 0.47508544892577603]"
